In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import config
from dataset import INAOEDataset, DFUData

In [ ]:
# Estimated Temperature (ET)
def estimated_temperature(hist: np.ndarray, cluster: np.ndarray):
    argmax = np.argmax(hist)
    if argmax < len(cluster) - 1:
        et_hist = hist[[argmax-1, argmax, argmax+1]]
        et_cluster = cluster[[argmax-1, argmax, argmax+1]]
    else:
        et_hist = hist[[argmax-1, argmax]]
        et_cluster = cluster[[argmax-1, argmax]]
    # et_cluster = cluster[[argmax-1, argmax, argmax+2]]
    return np.sum(et_hist * et_cluster) / np.sum(et_hist)

# Estimated Temperature difference (ETD)
def estimated_temperature_difference(left_et_angiosome, right_et_angiosome):
    return np.abs(left_et_angiosome - right_et_angiosome)

# Hot Pot Estimator
def hot_spot_estimator(hist:np.ndarray, cluster:np.ndarray):
    argmax = np.argmax(hist)
    et = estimated_temperature(hist, cluster)
    return np.abs(cluster[argmax] - et)

In [ ]:
from scipy.stats import skew, kurtosis

def get_metrics(dataset, data:DFUData):
    subjects = dataset.get_subjects_names()
    mean_data = []
    std_data = []
    max_data = []
    min_data = []
    skew_data = []
    kurtosis_data = []
    expected_temerature_data = []
    hot_spot_data = []
    metrics = []
    nrt_c1, nrt_c2, nrt_c3, nrt_c4, nrt_c5, nrt_c6, nrt_c7, nrt_c8, nrt_c9, nrt_c10 = [], [], [], [], [], [], [], [], [], []

    fulldata = None
    for idx in range(len(dataset)):
        # subject_data = dataset.get_subject(idx, data).to_numpy()
        subject_data = dataset.get_subject(idx, data)
        subject_data = subject_data[subject_data>18] # Remove background, temperatures lower than 18 are considered outliers!
        mean_data.append(subject_data.mean())
        std_data.append(subject_data.std())
        max_data.append(subject_data.max())
        min_data.append(subject_data.min())
        skew_data.append(skew(subject_data))
        kurtosis_data.append(kurtosis(subject_data))

        hist, _ = np.histogram(subject_data, bins=config.EXPECTED_VALUE_INTERVAL)
    
        # NRTClass
        nrt_class = hist / hist.sum()
        nrt_c1.append(nrt_class[0])
        nrt_c2.append(nrt_class[1])
        nrt_c3.append(nrt_class[2])
        nrt_c4.append(nrt_class[3])
        nrt_c5.append(nrt_class[4])
        nrt_c6.append(nrt_class[5])
        nrt_c7.append(nrt_class[6])
        nrt_c8.append(nrt_class[7])
        nrt_c9.append(nrt_class[8])
        nrt_c10.append(nrt_class[9])
    
        # Expected Temperature (ET) and Hot Spot Estimator
        expected_temerature_data.append(estimated_temperature(hist, config.EXPECTED_VALUE_CLUSTER))
        hot_spot_data.append(hot_spot_estimator(hist, config.EXPECTED_VALUE_CLUSTER))

    metrics = {'Subject': subjects,
        '{}_mean'.format(data.name): mean_data, 
        '{}_std'.format(data.name): std_data,
        '{}_min'.format(data.name): min_data,
        '{}_max'.format(data.name): max_data,
        '{}_skew'.format(data.name): skew_data,
        '{}_kurtosis'.format(data.name): kurtosis_data,
        '{}_NRT_C1'.format(data.name): nrt_c1,
        '{}_NRT_C2'.format(data.name): nrt_c2,
        '{}_NRT_C3'.format(data.name): nrt_c3,
        '{}_NRT_C4'.format(data.name): nrt_c4,
        '{}_NRT_C5'.format(data.name): nrt_c5,
        '{}_NRT_C6'.format(data.name): nrt_c6,
        '{}_NRT_C7'.format(data.name): nrt_c7,
        '{}_NRT_C8'.format(data.name): nrt_c8,
        '{}_NRT_C9'.format(data.name): nrt_c9,
        '{}_NRT_C10'.format(data.name): nrt_c10,
        '{}_HSE'.format(data.name): hot_spot_data,
        '{}_ET'.format(data.name): expected_temerature_data,
    }

    return pd.DataFrame(metrics)

In [ ]:
class IACTECDataset():
    def __init__(self, root_path: str, data: DFUData):
        self.root_path = root_path
        self.data = data

        angiosom = list(filter(lambda x: x.endswith(data.name), os.listdir(root_path)))[0]
        angiosom = os.path.join(root_path, angiosom)
        self.subjects = sorted(list(map(lambda x: os.path.join(angiosom, x), os.listdir(angiosom))))
        self.subjects = list(filter(lambda x: x.endswith('.txt'), self.subjects))

    def __len__(self):
        return len(self.subjects)

    def get_subject(self, idx: int, data: DFUData = None):
        return np.loadtxt(self.subjects[idx])

    def get_subjects_names(self):
        if self.data == DFUData.L or self.data == DFUData.R:
            return list(map(lambda x: x.split('/')[-1].split('.')[0], self.subjects))
        return list(map(lambda x: os.path.basename(x)[6:12], self.subjects))

In [ ]:
save_path = 'data/dataset/IACTEC/'

# MPA

In [ ]:
l_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.L_MPA)
r_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.R_MPA)

In [ ]:
len(l_dataset)

In [ ]:
l_metrics = get_metrics(l_dataset, DFUData.L_MPA)
r_metrics = get_metrics(r_dataset, DFUData.R_MPA)

mpa_metrics = pd.merge(l_metrics, r_metrics, on='Subject')
ETD = estimated_temperature_difference(mpa_metrics['L_MPA_ET'], mpa_metrics['R_MPA_ET'])
mpa_metrics['MPA_ETD'] = ETD

mpa_metrics.to_csv(os.path.join(save_path, 'MPA/MPA_metrics.csv'), index=False)

# MCA

In [ ]:
l_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.L_MCA)
r_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.R_MCA)

In [ ]:
l_metrics = get_metrics(l_dataset, DFUData.L_MCA)
r_metrics = get_metrics(r_dataset, DFUData.R_MCA)

mca_metrics = pd.merge(l_metrics, r_metrics, on='Subject')
ETD = estimated_temperature_difference(mca_metrics['L_MCA_ET'], mca_metrics['R_MCA_ET'])
mca_metrics['MCA_ETD'] = ETD

mca_metrics.to_csv(os.path.join(save_path, 'MCA/MCA_metrics.csv'), index=False)

# LCA

In [ ]:
l_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.L_LCA)
r_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.R_LCA)

In [ ]:
l_metrics = get_metrics(l_dataset, DFUData.L_LCA)
r_metrics = get_metrics(r_dataset, DFUData.R_LCA)

lca_metrics = pd.merge(l_metrics, r_metrics, on='Subject')
ETD = estimated_temperature_difference(lca_metrics['L_LCA_ET'], lca_metrics['R_LCA_ET'])
lca_metrics['LCA_ETD'] = ETD

lca_metrics.to_csv(os.path.join(save_path, 'LCA/LCA_metrics.csv'), index=False)

# LPA

In [ ]:
l_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.L_LPA)
r_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.R_LPA)

In [ ]:
l_metrics = get_metrics(l_dataset, DFUData.L_LPA)
r_metrics = get_metrics(r_dataset, DFUData.R_LPA)

lpa_metrics = pd.merge(l_metrics, r_metrics, on='Subject')
ETD = estimated_temperature_difference(lpa_metrics['L_LPA_ET'], lpa_metrics['R_LPA_ET'])
lpa_metrics['LPA_ETD'] = ETD

lpa_metrics.to_csv(os.path.join(save_path, 'LPA/LPA_metrics.csv'), index=False)

# Full Foot

In [ ]:
l_lca_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.L_LCA)
l_lpa_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.L_LPA)
l_mca_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.L_MCA)
l_mpa_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.L_MPA)

datasets = [l_lca_dataset, l_lpa_dataset, l_mca_dataset, l_mpa_dataset]
subjects_name = l_lca_dataset.get_subjects_names()
save_dir = 'natalia/dataset/IACTEC/00_Output_L/'
for subject_idx in range(len(l_lca_dataset)):
    data = []
    for angiosom in datasets:
        data.append(angiosom.get_subject(subject_idx, DFUData.L))

    data = np.concatenate(data, axis=0)
    np.savetxt(os.path.join(save_dir, subjects_name[subject_idx] + '.txt'), data) 

In [ ]:
r_lca_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.R_LCA)
r_lpa_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.R_LPA)
r_mca_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.R_MCA)
r_mpa_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.R_MPA)

datasets = [r_lca_dataset, r_lpa_dataset, r_mca_dataset, r_mpa_dataset]
subjects_name = r_lca_dataset.get_subjects_names()
save_dir = 'natalia/dataset/IACTEC/00_Output_R/'
for subject_idx in range(len(r_lca_dataset)):
    data = []
    for angiosom in datasets:
        data.append(angiosom.get_subject(subject_idx, DFUData.L))

    data = np.concatenate(data, axis=0)
    np.savetxt(os.path.join(save_dir, subjects_name[subject_idx] + '.txt'), data) 

In [ ]:
l_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.L)
r_dataset = IACTECDataset('natalia/dataset/IACTEC/', DFUData.R)

In [ ]:
l_metrics = get_metrics(l_dataset, DFUData.L)
r_metrics = get_metrics(r_dataset, DFUData.R)

full_metrics = pd.merge(l_metrics, r_metrics, on='Subject')
ETD = estimated_temperature_difference(full_metrics['L_ET'], full_metrics['R_ET'])
full_metrics['Foot_ETD'] = ETD

full_metrics.to_csv(os.path.join(save_path, 'Full/Full_metrics.csv'), index=False)

# Merge

In [ ]:
from functools import reduce 

feature_dataset_df = reduce(lambda left,right: pd.merge(left,right,on='Subject'), [mpa_metrics, lca_metrics, lpa_metrics, mca_metrics, full_metrics])

# group = 'dm' if dm_group else 'cg'
feature_dataset_df.to_csv(os.path.join(save_path, '{}_metrics.csv'.format('iactec')), index=False)

In [ ]:
def temperature_cg_values(angiosomes: DFUData):
    '''
        Returns the control temperature for the given angiosomes based on the paper
    '''
    data = [-1, -1, 26.1, 26.1, 25.7, 25.7, 26.4, 26.4, 25.8, 25.8]
    if isinstance(angiosomes, list):
        return np.array(list(map(lambda x: data[x.value], angiosomes)))
    else:
        return data[angiosomes.value]

def thermal_change_index(features_df: pd.DataFrame):
    feet_angiosomes = [ [DFUData.L_LCA, DFUData.L_LPA, DFUData.L_MCA, DFUData.L_MPA],
                        [DFUData.R_LCA, DFUData.R_LPA, DFUData.R_MCA, DFUData.R_MPA] ]

    tci = []
    for foot_angiosomes in feet_angiosomes:
        columns = list(map(lambda x: '{}_mean'.format(x.name), foot_angiosomes))
        control_temperature = temperature_cg_values(foot_angiosomes)
        tci.append((np.abs(control_temperature - features_df[columns].to_numpy())).mean(axis=1))
    
    return tci

left_tci, right_tci = thermal_change_index(feature_dataset_df)

feature_dataset_df['L_TCI'] = left_tci
feature_dataset_df['R_TCI'] = right_tci    

group = 'iactec'
feature_dataset_df.to_csv(os.path.join(save_path, '{}_metrics.csv'.format(group)), index=False)

In [ ]:
feature_dataset_df

# Testing

In [ ]:
import pandas as pd

iactec_df = pd.read_csv('data/dataset/IACTEC/iactec_metrics.csv', index_col=0)
inaoe_df = pd.read_csv('data/dataset/dfu_features_dataset.csv', index_col=0)

In [ ]:
features_dataset = pd.concat([inaoe_df,iactec_df], axis=0)
features_dataset.to_csv('data/dataset/dfu_features_dataset.csv')

# Generate Labels

In [ ]:
cg_group = inaoe_df.filter(regex='^CG', axis=0).to_numpy()
dm_group = inaoe_df.filter(regex='^DM', axis=0).to_numpy()

In [ ]:
import numpy as np
cg_group_labels = np.zeros(len(cg_group))
dm_group_labels = np.ones(len(dm_group))
iactec_labels = np.zeros(len(iactec_df))
y = np.concatenate([cg_group_labels, dm_group_labels, iactec_labels], axis=0)

# create dataframe with subjects and labels
labels_df = pd.DataFrame({'Subject': features_dataset.index, 'Label': y.astype(int)})
labels_df.to_csv('data/dataset/dfu_labels_dataset.csv', index=False)